In [1]:
import random
import random as rand
from random import randint
import numpy as np
import tensorflow as tf
#bce = tf.keras.losses.BinaryCrossentropy()
from  sklearn import  datasets
iris=datasets.load_iris()
X = iris.data
y = iris.target

2022-07-06 22:59:39.350189: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-06 22:59:39.350241: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
dims = (4,4,4,4,3)
def Net(dims, inputs):
    net = []
    I = 0
    for i in dims:
        l =  0.01 * np.random.randn(inputs, i)
        net.append(l)
        #print(l)
        inputs = i
        I+=1
    return net


In [3]:
def forward(net, inputs):
    #print(net)
    for i in range(len(net)):
        if i != (len(net)-1):
            inputs = relu(np.dot(inputs, net[i]))
        else:
            inputs = softmax(np.dot(inputs, net[i])) 
        #print(inputs.shape)
        #inputs = relu(np.dot(net[i], inputs))
    return inputs

In [4]:
def relu(x):
    for i in range(len(x)):
        if x[i] > 0:
            pass
        else:
            pass
            #x[i] = 0
    return x

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def softmax(vector):
	e = np.exp(vector)
	return e / e.sum()

In [5]:
def flatten(net):
    front = np.array(net[:len(net)-1])
    flat = front.flatten()
    out = net[-1].flatten()
    network = np.concatenate((flat, out), axis=0)
    #print(network.shape)
    return network

def reform(dims, net):
    sub = dims[-1] * dims[-2]
    #print(net)
    output = net[-sub:]
    rest = net[:-sub]
    #print(rest)
    network = rest.reshape(dims[:-2])
    output = output.reshape(4,3)
    #print(np.array([network, output]))
    layers = []
    for i in range(len(network)+1):
        if i!=4:
            layers.append(network[i])
        else:
            layers.append(output)
        #print(i)
    return layers

In [6]:
# call population to get list of numpy arrays
pops = []
for i in range(50):
    pops.append(Net(dims, 4))
#SortedPop = pops
print(len(pops))

50


In [7]:
def accuracy(X, y, net):
    correct = 0
    for i in range(round(len(y)/1)):
        #print(best)
        #print(len(net))
        #print(net)
        #print(forward(net, X[i]))
        if y[i] - np.argmax(forward(net, X[i])) == 0:
            correct += 1
    accuracy = correct/len(y)
    return accuracy

In [8]:
#sort population based its objective score
def objective(pops, X, y):
    SortedPop = []
    bestE = 0
    bestI = 0
    best = pops[0]
    for i in range(len(pops)):
        #print(best)
        if bestE < accuracy(X, y, pops[i]):
            bestI = i
            best = pops[i]
            bestE = accuracy(X, y, pops[i])
    pops.pop(bestI)
    return best, bestE, pops

SortedPop = []
for i in range(len(pops)):
    best, score, pops = objective(pops, X, y)
    #print(score)
    #isinstance(best, list)
    if isinstance(best, list):
        SortedPop.append(best)
        #print(np.array(best[:-1]).shape)
#print('first')    
#print(SortedPop[0])
print(len(SortedPop))
pops = SortedPop

50


In [9]:

def crossover(p1, p2, r_cross):
	# children are copies of parents by default
	c1, c2 = p1.copy(), p2.copy()
	# check for recombination
	if random.uniform(0, 1) < r_cross:
		# select crossover point that is not on the end of the string
		pt = randint(1, 76-2)
		#print(len(flatten(c1)))
		# perform crossover
		c1 = np.concatenate((flatten(c1)[:pt], flatten(c2)[pt:]), axis=0)
		#c1 = p1[:pt] + p2[pt:]
		c2 = np.concatenate((flatten(c2)[:pt], flatten(c1)[pt:]), axis=0)
		#c2 = p2[:pt] + p1[pt:]
		#print(c1.shape)
		#print(c1.shape)
		return [c1, c2]
	return [flatten(c1), flatten(c2)]
'''
def crossover(p1, p2, r_cross):
    #print('cross')
    #if random.uniform(0, 1) < r_cross:
    pt = randint(1, 76-2)
    #print(pt)
    c1 = np.concatenate((flatten(p1)[:pt], flatten(p2)[pt:]), axis=0)
    #c1 = p1[:pt] + p2[pt:]
    c2 = np.concatenate((flatten(p2)[:pt], flatten(p1)[pt:]), axis=0)
    return [c1, c2]
'''

"\ndef crossover(p1, p2, r_cross):\n    #print('cross')\n    #if random.uniform(0, 1) < r_cross:\n    pt = randint(1, 76-2)\n    #print(pt)\n    c1 = np.concatenate((flatten(p1)[:pt], flatten(p2)[pt:]), axis=0)\n    #c1 = p1[:pt] + p2[pt:]\n    c2 = np.concatenate((flatten(p2)[:pt], flatten(p1)[pt:]), axis=0)\n    return [c1, c2]\n"

In [10]:
def mutation(bitstring, r_mut):
	for i in range(len(bitstring)):
		# check for a mutation
		if random.uniform(0, 1) < r_mut:
			# flip the bit
			bitstring[i] = np.random.rand() * .01
			#print('boink')
#	return bitstring

In [11]:
def genetic_algorithm(gens, pops, X, y):
    #pops = population(24, 12)
    #print(len(pops))
    SortedPop = [pops[0]]
    g = 0
    #print(accuracy(X, y, SortedPop[0]))
    while accuracy(X, y, SortedPop[0]) < .9:
        SortedPop = []
        for j in range(len(pops)):
            best, score, pops = objective(pops, X, y)
            SortedPop.append(best)
            #print(np.array(SortedPop[0][:-1]).shape)
        
        #children = []
        SortedPop = SortedPop[:50]
        for i in range(0, len(SortedPop), 2):
            p1, p2 = SortedPop[i], SortedPop[i+1]
            for c in crossover(p1,p2, .8):
                mutation(c, .2)
                SortedPop.append(reform((4,4,4,4,3),c))
        g += 1
        print(g)
        pops = SortedPop
        print(accuracy(X, y, SortedPop[0]))
    return SortedPop[0]
    

In [ ]:
#print(np.array(crossover(pops[0], pops[1], .8)[0][:-1]).shape)
genetic_algorithm(100, pops, X, y)

1
0.6666666666666666
2
0.6666666666666666
3
0.8
4
0.8
5
0.8
6
0.8
7
0.8
